In [1]:
from utils.text_preprocessing import full_preprocess
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import fasttext

In [2]:
df = pd.read_csv('data/train_spam.csv')

In [3]:
df = full_preprocess(df)

In [4]:
df

,text_type,text,lemmatized_tokens,preprocessed_text
0,ham,"[make, sure, alex, knows, birthday, fifteen, m...","[make, sure, alex, know, birthday, fifteen, mi...",make sure alex know birthday fifteen minute fa...
1,ham,"[resume, john, lavorato, thanks, vince, get, m...","[resume, john, lavorato, thanks, vince, get, m...",resume john lavorato thanks vince get moving r...
2,spam,"[plzz, visit, website, moviesgodml, get, movie...","[plzz, visit, website, moviesgodml, get, movie...",plzz visit website moviesgodml get movie free ...
3,spam,"[urgent, mobile, number, awarded, £2000, prize...","[urgent, mobile, number, awarded, £2000, prize...",urgent mobile number awarded £2000 prize guara...
4,ham,"[overview, hr, associates, analyst, project, p...","[overview, hr, associate, analyst, project, pe...",overview hr associate analyst project per davi...
...,...,...,...,...
16273,spam,"[interested, binary, options, trading, may, co...","[interested, binary, option, trading, may, con...",interested binary option trading may continue ...
16274,spam,"[dirty, pictureblyk, aircel, thanks, valued, m...","[dirty, pictureblyk, aircel, thanks, valued, m...",dirty pictureblyk aircel thanks valued member ...
16275,ham,"[could, g, mon, 1635465, sep, 1635465, david, ...","[could, g, mon, 1635465, sep, 1635465, david, ...",could g mon 1635465 sep 1635465 david rees wro...
16276,ham,"[insta, reels, par, 80, गंद, bhara, pada, hai,...","[insta, reel, par, 80, गंद, bhara, pada, hai, ...",insta reel par 80 गंद bhara pada hai 👀 kuch bh...


In [5]:
df['text_type'] = df['text_type'].apply(lambda x: '__label__' + x)

In [6]:
df

,text_type,text,lemmatized_tokens,preprocessed_text
0,__label__ham,"[make, sure, alex, knows, birthday, fifteen, m...","[make, sure, alex, know, birthday, fifteen, mi...",make sure alex know birthday fifteen minute fa...
1,__label__ham,"[resume, john, lavorato, thanks, vince, get, m...","[resume, john, lavorato, thanks, vince, get, m...",resume john lavorato thanks vince get moving r...
2,__label__spam,"[plzz, visit, website, moviesgodml, get, movie...","[plzz, visit, website, moviesgodml, get, movie...",plzz visit website moviesgodml get movie free ...
3,__label__spam,"[urgent, mobile, number, awarded, £2000, prize...","[urgent, mobile, number, awarded, £2000, prize...",urgent mobile number awarded £2000 prize guara...
4,__label__ham,"[overview, hr, associates, analyst, project, p...","[overview, hr, associate, analyst, project, pe...",overview hr associate analyst project per davi...
...,...,...,...,...
16273,__label__spam,"[interested, binary, options, trading, may, co...","[interested, binary, option, trading, may, con...",interested binary option trading may continue ...
16274,__label__spam,"[dirty, pictureblyk, aircel, thanks, valued, m...","[dirty, pictureblyk, aircel, thanks, valued, m...",dirty pictureblyk aircel thanks valued member ...
16275,__label__ham,"[could, g, mon, 1635465, sep, 1635465, david, ...","[could, g, mon, 1635465, sep, 1635465, david, ...",could g mon 1635465 sep 1635465 david rees wro...
16276,__label__ham,"[insta, reels, par, 80, गंद, bhara, pada, hai,...","[insta, reel, par, 80, गंद, bhara, pada, hai, ...",insta reel par 80 गंद bhara pada hai 👀 kuch bh...


In [7]:
df.drop(['text', 'lemmatized_tokens'], inplace=True, axis=1)

In [8]:
df_train, df_test = train_test_split(df, test_size=0.33)

In [9]:
df_train.to_csv('data/train.txt', index = False, sep = ' ', header = None, quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")
df_test.to_csv('data/test.txt', index = False, sep = ' ', header = None, quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")

In [10]:
model = fasttext.train_supervised('data/train.txt', lr=.1, epoch=100, wordNgrams=10)

In [11]:
model.test('data/test.txt')

(5372, 0.9482501861504096, 0.9482501861504096)

In [12]:
preds = np.vstack(model.predict(df_test['preprocessed_text'].to_list())[1]).ravel()
preds[preds > 1] = 1

In [13]:
labels_test = df_test['text_type'].apply(lambda x: {'__label__ham': 1, '__label__spam': 0}[x])

In [14]:
roc_auc_score(labels_test, preds)

0.7845526388729498